In [87]:
from sklearn.preprocessing import MinMaxScaler
from nltk.corpus import stopwords
from sklearn import svm
import xgboost as xgb
from sklearn import svm
from sklearn.metrics import f1_score, accuracy_score
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import roc_auc_score

import pandas as pd
import scipy
import sklearn
from sklearn import *
import numpy as np
from nltk import *
import os
import statistics
import editdistance
import itertools  
import re



In [88]:
full_train_df = pd.read_csv('quora_question_pairs/quora_train.csv')
#train_df, test_df = sklearn.model_selection.train_test_split(full_train_df, test_size=0.1,random_state=123)


In [10]:
#Convert all in string
def cast_list_as_strings(mylist):
    """
    return a list of strings
    """
    #assert isinstance(mylist, list), f"the input mylist should be a list it is {type(mylist)}"
    mylist_of_strings = []
    for x in mylist:
        mylist_of_strings.append(str(x))

    return mylist_of_strings

#Removing Stops Words
#from nltk.corpus import stopwords
#nltk.download('stopwords')

def remove_stopwords(doc):
    stops     = set(stopwords.words("english"))
    token_doc = set(word_tokenize(doc))
    token_doc = list(token_doc - stops)
    return " ".join(token_doc)

#Adding len Words in common feat
def len_common(q1, q2):
    q1 = set(word_tokenize(q1)) ; q2 = set(word_tokenize(q2))
    return len(q1.intersection(q2))

#Adding len common words in common feat
def len_not_common(q1,q2):
    q1 = set(word_tokenize(q1)) ; q2 = set(word_tokenize(q2))
    return len(q1 ^ q2)

#Adding mean distance between common words 
def mean_dist_not_com(q1,q2):
    q1 = set(word_tokenize(q1)) ; q2 = set(word_tokenize(q2))
    not_comm1 = (q1 ^ q2) - q1
    if len(not_comm1)==0 : not_comm1={''}
    not_comm2 = (q1 ^ q2) - q2
    if len(not_comm2)==0 : not_comm2={''}
    return statistics.mean([editdistance.eval(i[0],i[1]) for i in itertools.product(not_comm1, not_comm2)])

def hasNumbers(inputString): return bool(re.search(r'\d', inputString)) 
def both_number(q1,q2): return hasNumbers(q1) *  hasNumbers(q2) 

In [ ]:
def preprocess(df, scaler):
    df = df.copy()
    df['question1_string'] = cast_list_as_strings(list(df["question1"]))
    df['question2_string'] = cast_list_as_strings(list(df["question2"]))
    
    df['question1_no_stops'] = [remove_stopwords(quest) for quest in df["question1_string"]]
    df['question2_no_stops'] = [remove_stopwords(quest) for quest in df["question2_string"]]
    
    df['len_q1'] = [len(s) for s in df['question1_string']] 
    df['len_q2'] = [len(s) for s in df['question2_string']]
    df['len_q1'] = scaler.fit_transform(df[['len_q1']])
    df['len_q2'] = scaler.fit_transform(df[['len_q2']])
    
    df['len_common'] = [len_common(df.loc[i,'question1_string'],df.loc[i,'question2_string']) for i in range(len(df))]
    df['len_common'] = scaler.fit_transform(df[['len_common']])

    df['len_not_common'] = [len_not_common(df.loc[i,'question1_string'],df.loc[i,'question2_string']) for i in range(len(df))]
    df['len_not_common'] = scaler.fit_transform(df[['len_not_common']])
    
    df['mean_dist_not_com'] = [mean_dist_not_com(df.loc[i,'question1_string'],df.loc[i,'question2_string']) for i in range(len(df))]
    df['mean_dist_not_com'] = scaler.fit_transform(df[['mean_dist_not_com']])
    
    df['both_number'] = [both_number(df.loc[i,'question1_string'],df.loc[i,'question2_string']) for i in range(len(df))]
    
    return df

In [54]:
df_full = pd.read_csv('data_preprocessed.csv')
df_full = df_full.dropna()

## BOW REPRESENTATION

In [89]:
def fit_on_q1_q2(df, model):
    q_list1 = list(df["question1_string"])
    q_list2 = list(df["question2_string"])
    all_questions = q_list1 + q_list2 
    model.fit(all_questions)
    return

def get_features_from_df(df, count_vectorizer):
    """
    returns a sparse matrix containing the features build by the count vectorizer.
    Each row should contain features from question1 and question2.
    """
    q1_casted =  cast_list_as_strings(list(df["question1"]))
    q2_casted =  cast_list_as_strings(list(df["question2"]))
    
    ############### Begin exercise ###################
    # what is kaggle                  q1
    # What is the kaggle platform     q2
    X_q1 = count_vectorizer.transform(q1_casted)
    X_q2 = count_vectorizer.transform(q2_casted)    
    X_q1q2 = scipy.sparse.hstack((X_q1,X_q2))
    ############### End exercise ###################

    return X_q1q2



In [97]:
    logistic = sklearn.linear_model.LogisticRegression(solver="liblinear",
                                                       random_state=123)

    
    train_df, test_df = sklearn.model_selection.train_test_split(df_full, test_size=0.05,random_state=123)
    train_df, val_df  = sklearn.model_selection.train_test_split(train_df, test_size=0.05,random_state=123)
    X_train_q1q2      = get_features_from_df(train_df, count_vectorizer)
    X_val_q1q2        = get_features_from_df(val_df, count_vectorizer)
    X_test_q1q2       = get_features_from_df(test_df, count_vectorizer)
    y_train           = train_df["is_duplicate"].values
    y_val             = val_df["is_duplicate"].values
    y_test            = test_df["is_duplicate"].values
    
    logistic.fit(X_train_q1q2, y_train)

                                                   
         

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=123, solver='liblinear', tol=0.0001, verbose=0,
                   warm_start=False)

In [102]:
xgb_model.fit(X_train_q1q2, y_train) 


/Users/pablogranatiero/opt/anaconda3/lib/python3.7/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[18:14:42] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:541: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[18:14:53] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.7, eta=0.3, gamma=0,
              gpu_id=-1, importance_type='gain', interaction_constraints='',
              learning_rate=0.1, max_delta_step=0, max_depth=50,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=80, n_jobs=4, num_parallel_tree=1,
              objective='binary:logistic', random_state=123, reg_alpha=4,
              reg_lambda=1, scale_pos_weight=1, silent=1, subsample=0.8,
              tree_method='exact', use_label_encoder=True,
              validate_parameters=1, verbosity=None)

In [104]:
xgb_model.predict_proba(X_train_q1q2)[:, 1]

array([0.25333562, 0.14860882, 0.42875916, ..., 0.39663774, 0.30302086,
       0.07505125], dtype=float32)

In [103]:
roc_auc_score(y_train, xgb_model.predict_proba(X_train_q1q2)[:, 1])

0.9279273390519733

In [92]:
def train_models(df, get_feat_model, vectorizer_func):
    
    logistic = sklearn.linear_model.LogisticRegression(solver="liblinear",
                                                       random_state=123)

    xgb_model = xgb.XGBClassifier(max_depth=50, n_estimators=80, 
                              learning_rate=0.1, colsample_bytree=.7, gamma=0, reg_alpha=4, 
                              objective='binary:logistic', eta=0.3, silent=1, subsample=0.8, random_state=123)

    #svm_model = svm.SVC()
    
    train_df, test_df = sklearn.model_selection.train_test_split(df, test_size=0.05,random_state=123)
    train_df, val_df  = sklearn.model_selection.train_test_split(train_df, test_size=0.05,random_state=123)
    X_train_q1q2      = get_feat_model(train_df, vectorizer_func)
    X_val_q1q2        = get_feat_model(val_df, vectorizer_func)
    X_test_q1q2       = get_feat_model(test_df, vectorizer_func)
    y_train           = train_df["is_duplicate"].values
    y_val             = val_df["is_duplicate"].values
    y_test            = test_df["is_duplicate"].values
    
    logistic.fit(X_train_q1q2, y_train)
    xgb_model.fit(X_train_q1q2, y_train) 
    #svm_model.fit(X_tr_q1q2, y_train)
                                                   
    logistic_train_acc = roc_auc_score(y_train, logistic.predict_proba(X_train_q1q2)[:, 1])                                                       
    logistic_val_acc   = roc_auc_score(y_train, logistic.predict_proba(X_val_q1q2)[:, 1])
    logistic_test_acc  = roc_auc_score(y_train, logistic.predict_proba(X_test_q1q2)[:, 1])
    print('logistic_train_acc:{}, logistic_val_acc:{}, logistic_test_acc:{}'.format(logistic_train_acc, logistic_val_acc, logistic_test_acc))
                                                   
    xgb_train_acc      = roc_auc_score(y_train, xgb_model.predict_proba(X_train_q1q2)[:, 1])
    xgb_val_acc        = roc_auc_score(y_val, xgb_model.predict_proba(X_val_q1q2)[:, 1])
    xgb_test_acc       = roc_auc_score(y_test, xgb_model.predict_proba(X_test_q1q2)[:, 1])
    print('xgb_train_acc:{}, xgb_val_acc:{}, xgb_test_acc:{}'.format(xgb_train_acc, xgb_val_acc, xgb_test_acc))
                                                   
    return [logistic_train_acc, logistic_val_acc, logistic_test_acc], [xgb_train_acc, xgb_val_acc, xgb_test_acc]
                                                       
                                                       
                                                       
                           
                                                       

In [93]:
count_vectorizer = sklearn.feature_extraction.text.CountVectorizer(ngram_range=(1,1))

fit_on_q1_q2(df_full, count_vectorizer)

In [ ]:
acc_logistic, acc_xgboost = train_models(df_full, get_features_from_df, count_vectorizer)

## TFIDF REPRESENTATION

In [62]:
def get_tfidf(df, tfidf, sim=False):
    q1_casted =  cast_list_as_strings(list(df["question1"]))
    q2_casted =  cast_list_as_strings(list(df["question2"]))
    
    tfidf_q1 = tfidf.transform(q1_casted)
    tfidf_q2 = tfidf.transform(q2_casted)
    tfidf_q1q2 = scipy.sparse.hstack((tfidf_q1,tfidf_q2))
    if sim == True:
        sims = []
        for i in range(len(q1_casted)):
            sims.append(cosine_similarity(tfidf_q1[i,:],tfidf_q2[i,:]))
        sims = np.reshape(sims, (len(q1_casted), 1))

        return scipy.sparse.hstack((tfidf_q1q2,sims)).tocsr() 
    else:
        return tfidf_q1q2.tocsr() 

In [57]:
tfidf = sklearn.feature_extraction.text.TfidfVectorizer()
fit_on_q1_q2(df_full, tfidf)


In [63]:
acc_logistic_tfidf, acc_xgboost_tfidf = train_models(df_full, get_tfidf, tfidf)

/Users/pablogranatiero/opt/anaconda3/lib/python3.7/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[20:32:13] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:541: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[20:32:29] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
logistic_train_acc:0.79214059014093, logistic_val_acc:0.7641116434076234, logistic_test_acc:0.765075439030423
xgb_train_acc:0.8791637191229951, xgb_val_acc:0.7719671763079887, xgb_test_acc:0.7753559945880965


### Adding cosine similarity

In [64]:
def get_tfidf(df, tfidf, sim=True):
    q1_casted =  cast_list_as_strings(list(df["question1"]))
    q2_casted =  cast_list_as_strings(list(df["question2"]))
    
    tfidf_q1 = tfidf.transform(q1_casted)
    tfidf_q2 = tfidf.transform(q2_casted)
    tfidf_q1q2 = scipy.sparse.hstack((tfidf_q1,tfidf_q2))
    if sim == True:
        sims = []
        for i in range(len(q1_casted)):
            sims.append(cosine_similarity(tfidf_q1[i,:],tfidf_q2[i,:]))
        sims = np.reshape(sims, (len(q1_casted), 1))

        return scipy.sparse.hstack((tfidf_q1q2,sims)).tocsr() 
    else:
        return tfidf_q1q2.tocsr() 

In [65]:
acc_logistic_tfidf_sim, acc_xgboost_tfidf_sim = train_models(df_full, get_tfidf, tfidf)

/Users/pablogranatiero/opt/anaconda3/lib/python3.7/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[21:17:45] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:541: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[21:18:08] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
logistic_train_acc:0.8256052973452976, logistic_val_acc:0.7962403665902937, logistic_test_acc:0.7995547860499629
xgb_train_acc:0.9179797452089911, xgb_val_acc:0.8042715469580926, xgb_test_acc:0.8025583271975577


### Adding features: len(questions)

In [ ]:
def add_a_column_feat(df, col_list, sparse_matrix):

    for col_q in col_list:
        feat_q = df[col_q].to_numpy().reshape(len(df[col_q]),1)
        sparse_matrix = scipy.sparse.hstack((sparse_matrix,feat_q)).tocsr()
    
    return sparse_matrix     

In [105]:
def train_models(df, get_feat_model, vectorizer_func, col_list):
    
    logistic = sklearn.linear_model.LogisticRegression(solver="liblinear",
                                                       random_state=123)

    xgb_model = xgb.XGBClassifier(max_depth=50, n_estimators=80, 
                              learning_rate=0.1, colsample_bytree=.7, gamma=0, reg_alpha=4, 
                              objective='binary:logistic', eta=0.3, silent=1, subsample=0.8, random_state=123)

    #svm_model = svm.SVC()
    
    train_df, test_df = sklearn.model_selection.train_test_split(df, test_size=0.05,random_state=123)
    train_df, val_df  = sklearn.model_selection.train_test_split(train_df, test_size=0.05,random_state=123)
    X_train_q1q2      = get_feat_model(train_df, vectorizer_func)
    X_val_q1q2        = get_feat_model(val_df, vectorizer_func)
    X_test_q1q2       = get_feat_model(test_df, vectorizer_func)
    
    X_train_q1q2      = add_a_column_feat_single(train_df, col_list, X_train_q1q2)
    X_val_q1q2        = add_a_column_feat_single(val_df, col_list, X_val_q1q2)
    X_test_q1q2       = add_a_column_feat_single(test_df, col_list, X_test_q1q2)
    
    y_train           = train_df["is_duplicate"].values
    y_val             = val_df["is_duplicate"].values
    y_test            = test_df["is_duplicate"].values
    
    logistic.fit(X_train_q1q2, y_train)
    xgb_model.fit(X_train_q1q2, y_train) 
    #svm_model.fit(X_tr_q1q2, y_train)
                                                   
    logistic_train_acc = roc_auc_score(y_train, logistic.predict_proba(X_train_q1q2)[:, 1])                                                       
    logistic_val_acc   = roc_auc_score(y_train, logistic.predict_proba(X_val_q1q2)[:, 1])
    logistic_test_acc  = roc_auc_score(y_train, logistic.predict_proba(X_test_q1q2)[:, 1])
    print('logistic_train_acc:{}, logistic_val_acc:{}, logistic_test_acc:{}'.format(logistic_train_acc, logistic_val_acc, logistic_test_acc))
                                                   
    xgb_train_acc      = roc_auc_score(y_train, xgb_model.predict_proba(X_train_q1q2)[:, 1])
    xgb_val_acc        = roc_auc_score(y_val, xgb_model.predict_proba(X_val_q1q2)[:, 1])
    xgb_test_acc       = roc_auc_score(y_test, xgb_model.predict_proba(X_test_q1q2)[:, 1])
    print('xgb_train_acc:{}, xgb_val_acc:{}, xgb_test_acc:{}'.format(xgb_train_acc, xgb_val_acc, xgb_test_acc))
                                                   
    return [logistic_train_acc, logistic_val_acc, logistic_test_acc], [xgb_train_acc, xgb_val_acc, xgb_test_acc]
        

In [77]:
acc_logistic_tfidf_sim_len, acc_xgboost_tfidf_sim_len = train_models(df_full, get_tfidf, tfidf, ['len_q1', 'len_q2'])


/Users/pablogranatiero/opt/anaconda3/lib/python3.7/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[22:21:13] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:541: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[22:21:35] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
logistic_train_acc:0.8261397459861537, logistic_val_acc:0.7966569464694855, logistic_test_acc:0.8002968093000248
xgb_train_acc:0.9294939453594195, xgb_val_acc:0.8049364103873642, xgb_test_acc:0.8059043933919123


### Adding features: mean_dist_not_com

In [78]:
acc_logistic_mean_dist_not_com, acc_xgboost_mean_dist_not_com = train_models(df_full, get_tfidf, tfidf, ['len_q1', 'len_q2', 'mean_dist_not_com'])


/Users/pablogranatiero/opt/anaconda3/lib/python3.7/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[22:48:34] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:541: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[22:48:58] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
logistic_train_acc:0.8266933799628353, logistic_val_acc:0.7978025411372631, logistic_test_acc:0.800939896116745
xgb_train_acc:0.9339173567042016, xgb_val_acc:0.806458369095475, xgb_test_acc:0.8071895533827116


In [80]:
acc_logistic_mean_dist_not_com = [0.8266933799628353, 0.7978025411372631, 0.800939896116745]
acc_xgboost_mean_dist_not_com =  [0.9339173567042016, 0.806458369095475, 0.8071895533827116]

In [85]:
hist = pd.DataFrame({'BOW_No_Feat_Logistic':acc_logistic,'BOW_No_Feat_XGB':acc_xgboost,
                     'TFIDF_No_Feat_Logistic':acc_logistic_tfidf,'TFIDF_No_Feat_XGB':acc_xgboost_tfidf,
                     'TFIDF_Cos_Sim_Logistic':acc_logistic_tfidf_sim,'TFIDF_Cos_Sim_XGB':acc_xgboost_tfidf_sim,
                     'TFIDF_Cos_Sim_Len_Logistic':acc_logistic_tfidf_sim_len,'TFIDF_Cos_Sim_Len_XGB':acc_xgboost_tfidf_sim_len,
                     'TFIDF_Cos_Sim_Len_Logistic':acc_logistic_tfidf_sim_len,'TFIDF_Cos_Sim_Len_XGB':acc_xgboost_tfidf_sim_len,
                     'TFIDF_mean_dist_not_com_Logistic':acc_logistic_mean_dist_not_com,'TFIDF_mean_dist_not_com_XGB':acc_xgboost_mean_dist_not_com,
                    }, index=['Train','Validation', 'Test'])
hist.to_csv('hist.csv')


In [86]:
hist

,BOW_No_Feat_Logistic,BOW_No_Feat_XGB,TFIDF_No_Feat_Logistic,TFIDF_No_Feat_XGB,TFIDF_Cos_Sim_Logistic,TFIDF_Cos_Sim_XGB,TFIDF_Cos_Sim_Len_Logistic,TFIDF_Cos_Sim_Len_XGB,TFIDF_mean_dist_not_com_Logistic,TFIDF_mean_dist_not_com_XGB
Train,0.809158,0.821916,0.792141,0.879164,0.825605,0.917980,0.826140,0.929494,0.826693,0.933917
Validation,0.754895,0.762478,0.764112,0.771967,0.796240,0.804272,0.796657,0.804936,0.797803,0.806458
Test,0.758595,0.766916,0.765075,0.775356,0.799555,0.802558,0.800297,0.805904,0.800940,0.807190
